[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mravanba/comp551-notebooks/blob/master/MLP.ipynb)

# Multilayer Perceptron (MLP)
Our goal here is to implement a two-layer neural network for binary classification, train it using gradient descent and use it to classify the Iris dataset.
Our model is
$$
\hat{y} = \sigma \left ( W \sigma \left ( V x \right ) \right)
$$
where we have $M$ hidden units and $D$ input features -- that is $w \in \mathbb{R}^{M}$, and $V \in \mathbb{R}^{M \times D}$. For simplicity here we do not include a bias parameter for each layer. Key to our implementation is the gradient calculation. We follow the notation used in the slides here.

In [1]:
import numpy as np
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
import time

# Data Preprocessing

In [2]:
# packaging it all into a function
def preprocess_fashion_mnist():
  import random as rand
   
   
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
  mean_mat = np.mean(x_train, axis=0)
  
  # centering the data by removing the pixel wise mean from every pixel in every image
  x_train_centered = x_train - mean_mat
  x_test_centered = x_test - mean_mat
  
  # normalizing the grayscale values to values in interval [0,1]
  x_train_normalized = x_train_centered/255.0
  x_test_normalized = x_test_centered/255.0

  #finally, flattening the data
  x_train = np.reshape(x_train_normalized, (60000,784))
  x_test = np.reshape(x_test_normalized, (10000, 784))
  #converting the test data to one hot encodings
  y_train = keras.utils.to_categorical(y_train, num_classes=10)
  y_test = keras.utils.to_categorical(y_test, num_classes=10)
  
  return x_train, y_train, x_test, y_test
x_train, y_train, x_test, y_test = preprocess_fashion_mnist()

4431872/4422102 [==============================] - 0s 0us/step


# Model Implementation - SoftMax


Activation functions

In [3]:
#activation functions
softmax1D = lambda z: np.exp(z)/float(sum(np.exp(z)))
softmax2D = lambda z: np.array([np.exp(i)/float(sum(np.exp(i))) for i in z])
# relu = lambda y: y[y <= 0]=0
def relu(x):
  alpha = 0.1
  x=np.array(x).astype(float)
  # x[x<=0]=0.1*x
  np.putmask(x, x<0, alpha*x)
  return x
def relu_grad(x):
  alpha = 0.1
  x=np.array(x).astype(float)
  x[x>0]=1
  x[x<=0]=alpha
  return x

Dimension Indications

Accuracy function

In [4]:
def evaluate_acc(pred, truth):
  counter =0
  
  for i in range(len(pred)):
    maxVal = np.where(pred[i] == np.amax(pred[i]))
    counter += 1 if maxVal == np.where(truth[i]==1) else 0
  return counter * 100.0 / float(len(pred))
  

Model

In [5]:
logistic = lambda z: 1./ (1 + np.exp(-z))

class MLP:
    
    def __init__(self, M = 128, num_classes = 10):
        self.M = M
        self.num_classes = num_classes
            
    def fit(self, x, y, optimizer):
        N,D = x.shape
        def gradient(x, y, params):
            w = params[0] # v.shape = (D, M), w.shape = (M)
            z = np.dot(x, w)
            yh = softmax2D(z)#N
            dy = yh - y #N
            train_acc = evaluate_acc(yh, y)
            dw = np.dot(x.T, dy)/N #M  
            dparams = [dw]
            # print(dparams)
            # print("yh shape:",len(yh[0]))
            return dparams ,train_acc
        
        # w = np.random.randn(self.M) * .01
        # v = np.random.randn(D,self.M) * .01
        initializer = keras.initializers.GlorotNormal()
        w = initializer(shape=(D, self.num_classes))
        # v = initializer(shape=(D, self.M))
        params0 = [w]
        self.params, train_accs, batch_train_accs = optimizer.run_mini_batch(gradient, x, y, params0) 
        return self, train_accs, batch_train_accs
    
    def predict(self, x):
        # print('self:',self)
        # print('self==None:',self==None)
        w = self.params[0]
        # print(w.shape)
        # z = relu(np.dot(x, w)) #N x M
        yh = softmax2D(np.dot(x, w))#N
        return yh

# Batch Implementation

In the implementation above we have used a list data structure to maintain model parameters and their gradients. Below I have modified the `GradientDescent` class to also work with a list of parameters. One sournce of confusion in the above implementation is the gradient calculation. While in the slides during the lectures 
we calculated the partial derivative for individual parameters, here, we use vector and matrix operations to calculate the derivative for *all* parameters. 

In [6]:
def mini_batcher(x, y, mini_batch_size):
  zipped = np.hstack( (x, y ) )
  np.random.shuffle(zipped)
  x_batches, y_batches = [], []
  mini_batches = []
  batch_num = x.shape[0] // mini_batch_size 
  for i in range(batch_num):
    x_batch = zipped[ i * mini_batch_size : (i+1) * mini_batch_size, :-10]
    y_batch = zipped[ i * mini_batch_size : (i+1) * mini_batch_size, -10:]
    mini_batches.append( ( x_batch, y_batch) )
    # mini_batches.append( ( x_batch, np.argmax(y_batch,axis=1)[:,None] ) )
  if x.shape[0] % mini_batch_size != 0:
    x_batch = zipped[ batch_num * mini_batch_size :, :-10]
    y_batch = zipped[ batch_num * mini_batch_size :, -10:]
    # print("Length of last mini-batch =", y_batch.shape[0])
    mini_batches.append( ( x_batch, y_batch ) )
    # mini_batches.append( ( x_batch, np.argmax(y_batch,axis=1) ) )
  # print(mini_batches[0])
  # print("yShape = ",y.shape)
  return mini_batches

# Gradient Descent 

---



In [7]:
class GradientDescent:
    
    def __init__(self, learning_rate=.001, max_iters=2e4, epsilon=1e-8, batch_size=32):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        while np.any(norms > self.epsilon) and t < self.max_iters:
            grad = gradient_fn(x, y, params)
            # print(grad[0].shape)
            # print(params[0].shape)
            for p in range(len(params)):
                params[p] -= self.learning_rate * grad[p]
            t += 1
            norms = np.array([np.linalg.norm(g) for g in grad])
        print(t)
        return params
    
    def run_mini_batch(self, gradient_fn, x, y, params, batch_size=32):
        train_acc, batch_train_acc, chunk = [], [], []
        norms = np.array([np.inf])
        t=1
        mini_batches = mini_batcher(x, y, batch_size)
        while np.any(norms > self.epsilon) and t < self.max_iters * len(mini_batches):
            x_temp, y_temp = mini_batches[t % ( len(mini_batches)-1 ) ][0], mini_batches[t % ( len(mini_batches)-1 ) ][1]
            grad, temp_acc = gradient_fn(x_temp, y_temp, params)
            for p in range(len(params)):
                params[p] -= self.learning_rate * grad[p]
            chunk.append(temp_acc)
            print(f"Epoch{t}:{temp_acc}%")
            train_acc.append( ( t, temp_acc ) )
            t += 1
            if t%len(mini_batches) == 2:
              batch_train_acc.append(np.mean(chunk))
              chunk = []
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params, train_acc, batch_train_acc

# MNIST DataSet

In [8]:
model = MLP(M=128, num_classes=10)
optimizer = GradientDescent(learning_rate=.004, max_iters=2000, batch_size=64)
y_pred, train_accs, batch_train_accs = model.fit(x_train, y_train, optimizer)

Epoch1:6.25%
Epoch2:15.625%
Epoch3:6.25%
Epoch4:3.125%
Epoch5:12.5%
Epoch6:15.625%
Epoch7:15.625%
Epoch8:0.0%
Epoch9:6.25%
Epoch10:6.25%
Epoch11:3.125%
Epoch12:12.5%
Epoch13:9.375%
Epoch14:3.125%
Epoch15:9.375%
Epoch16:9.375%
Epoch17:3.125%
Epoch18:15.625%
Epoch19:15.625%
Epoch20:9.375%
Epoch21:9.375%
Epoch22:12.5%
Epoch23:21.875%
Epoch24:18.75%
Epoch25:12.5%
Epoch26:3.125%
Epoch27:12.5%
Epoch28:9.375%
Epoch29:9.375%
Epoch30:9.375%
Epoch31:3.125%
Epoch32:9.375%
Epoch33:9.375%
Epoch34:12.5%
Epoch35:9.375%
Epoch36:3.125%
Epoch37:3.125%
Epoch38:6.25%
Epoch39:3.125%
Epoch40:9.375%
Epoch41:12.5%
Epoch42:15.625%
Epoch43:6.25%
Epoch44:12.5%
Epoch45:9.375%
Epoch46:3.125%
Epoch47:3.125%
Epoch48:6.25%
Epoch49:9.375%
Epoch50:15.625%
Epoch51:6.25%
Epoch52:9.375%
Epoch53:3.125%
Epoch54:9.375%
Epoch55:0.0%
Epoch56:6.25%
Epoch57:6.25%
Epoch58:12.5%
Epoch59:9.375%
Epoch60:3.125%
Epoch61:3.125%
Epoch62:3.125%
Epoch63:15.625%
Epoch64:15.625%
Epoch65:15.625%
Epoch66:15.625%
Epoch67:6.25%
Epoch68:12.5%
Ep

KeyboardInterrupt: ignored

In [ ]:
print("Number of full training batch iterations:",len(batch_train_accs))
print("Accuracies per training batch:")
for i in batch_train_accs:
  print(i)
y_test_pred = model.predict(x_test)
print("--------")
print("final accuracy")
print(evaluate_acc(y_test_pred, y_test))

In [ ]:
print("final accuracy")
print(evaluate_acc(y_test_pred, y_test))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(batch_train_accs, range(len(batch_train_accs)), c='r', marker='x')
ax.legend()
ax.grid(True)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(range(len(batch_train_accs)), batch_train_accs, c='r', marker='x')
ax.legend()
ax.grid(True)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
def hyper_tuning(x_train, y_train):
  from sklearn.model_selection import KFold
  import pandas as pd
  kf = KFold(5)
  acc_vals = []
  # hidden_units = [64, 128, 256, 512]
  # activations = [relu] #,leaky_relu, tanh ]
  learning_rate = [0.001, 0.002, 0.004]
  batch_size = [16, 32, 64]
  for btch in batch_size:
    print('batchsize:',btch)
    for lr in learning_rate:
      print('learningrate:',lr)
      optimizer = GradientDescent(learning_rate = lr, batch_size=btch)
      # for activ in activations:
      # for hu in hidden_units:   
      avg_acc = 0;       
      # print(f"for M=128, nonlinearity={activ}, lr={lr}, batch size={btch}.")
      start = time.time()
      for k, (train, test) in enumerate(kf.split(x_train, y_train)):
          print('k:',k)
          temp_model = MLP(M=128)
          # print(y_train[train].shape)
          # print('model is None:',temp_model==None)
          temp_model.fit(x_train[train], y_train[train], optimizer)
          # print('model is None:',temp_model==None)
          y_test_pred = temp_model.predict(x_train[test])
          temp_acc = evaluate_acc(y_test_pred, y_train[test])
          # 
          avg_acc += temp_acc
      avg_acc = avg_acc/5
      # print(f"{avg_acc} accuracy")
      acc_vals.append(avg_acc)
      end = time.time()
      print('time elapsed:',(end-start)/60/60,"hrs")
      print('acc:',avg_acc)
  data = {'learningRate' : [0.001, 0.002, 0.004, 0.001, 0.002, 0.004, 0.001, 0.002, 0.004], 
          'batchSize':[16, 16, 16, 32, 32, 32, 64, 64, 64],
          'accuracies': acc_vals
          }
  acc = pd.DataFrame(data)
  print(acc)
  return acc

In [ ]:
hyper_tuning(x_train=x_train[:2000], y_train=y_train[:2000])

# **Plot Hyperparameter Accuracies**

In [ ]:
# graph hyperparameter curves
learningRates= [0.001,0.002,0.004]
batch16Acc = [80.5,81.85,82.25] 
batch32Acc = [79.35,81.6,82.3]
batch64Acc = [79.15,81.2,82.4]

plt.plot(learningRates, batch16Acc)
plt.plot(learningRates, batch32Acc)
plt.plot(learningRates, batch64Acc)

plt.xlabel('Learning Rate')
plt.ylabel('Accuracy %')
plt.xticks(learningRates)
plt.legend(['16 Mini-Batch', '32 Mini-Batch', '64 Mini-Batch'])
plt.show()